In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score
import pickle


In [2]:
model_filepath = 'best_model.pkl'

with open(model_filepath, 'rb') as f:
    loaded_model = pickle.load(f)

In [3]:
def LOAN_APPROVER_MODEL(sample):
    sample_data = pd.DataFrame(sample,index=[0])
    # Calculating features
    sample_data['Loan_by_Income'] = sample_data['loan_amount'] / sample_data['income_annum']
    sample_data['Income_perDependent'] = sample_data['income_annum'] / (sample_data['no_of_dependents'] + 1)
    sample_data['Loan_by_Income_perDependent'] = sample_data['loan_amount'] / sample_data['Income_perDependent']

    sample_data['Loan_Per_Year'] = sample_data['loan_amount'] / sample_data['loan_term']
    sample_data['Loan_Per_Year_by_income_annum'] = sample_data['Loan_Per_Year'] / sample_data['income_annum']
    sample_data['Loan_Per_Year_by_Income_perDependent'] = sample_data['Loan_Per_Year'] / sample_data['Loan_by_Income_perDependent']

    sample_data['Total_Assets'] = (
        sample_data['residential_assets_value'] +
        sample_data['commercial_assets_value'] +
        sample_data['luxury_assets_value'] +
        sample_data['bank_asset_value']
    )
    sample_data['Real_Estate_Assets'] = sample_data['residential_assets_value'] + sample_data['commercial_assets_value']
    sample_data['Loan_by_Assests'] = sample_data['loan_amount'] / sample_data['Total_Assets']
    sample_data['Total_Income_Loan_Duration'] = sample_data['income_annum'] * sample_data['loan_term']
    sample_data['Loan_by_Total_income'] = sample_data['loan_amount'] / sample_data['Total_Income_Loan_Duration']
    sample_data['Loan_by_Assets_Total_income'] = sample_data['loan_amount'] / (sample_data['Total_Income_Loan_Duration'] + sample_data['Total_Assets'])
    sample_data['Loan_by_RealEstateAssets_Total_income'] = sample_data['loan_amount'] / (sample_data['Total_Income_Loan_Duration'] + sample_data['Real_Estate_Assets'])
    sample_data['Loan_by_RealEstateAssets_BankAssets_Total_income'] = sample_data['loan_amount'] / (sample_data['Total_Income_Loan_Duration'] + sample_data['Real_Estate_Assets'] + sample_data['bank_asset_value'])
    sample_data['Loan_by_TotalIncomeAsset_per_dependent'] = (
        sample_data['loan_amount'] * (sample_data['no_of_dependents'] + 1) /
        (sample_data['Total_Assets'] + sample_data['Total_Income_Loan_Duration'])
    )

    sample_data['Luxury_assets_by_Total_assets'] = sample_data['luxury_assets_value'] / sample_data['Total_Assets']
    sample_data['Liquidity_Ratio'] = sample_data['bank_asset_value'] / sample_data['Total_Assets']
    
    features=['Loan_by_Assests', 'Loan_by_Income', 'Loan_by_Total_income',
       'Loan_Per_Year_by_income_annum', 'Loan_by_Assets_Total_income',
       'Loan_by_RealEstateAssets_BankAssets_Total_income',
       'Luxury_assets_by_Total_assets', 'Liquidity_Ratio',
       'Loan_by_RealEstateAssets_Total_income',
       'Loan_by_Income_perDependent', 'commercial_assets_value',
       'residential_assets_value',
       'Loan_by_TotalIncomeAsset_per_dependent', 'Real_Estate_Assets',
       'Loan_Per_Year_by_Income_perDependent', 'Total_Assets',
       'luxury_assets_value', 'Total_Income_Loan_Duration',
       'Income_perDependent', 'bank_asset_value', 'Loan_Per_Year',
       'loan_amount', 'income_annum','cibil_score']
    
    if loaded_model.predict(sample_data[features])[0]==1:
        return 'Approved'
    else:
        return 'Rejected'
    
    

In [4]:
# Sample input data
sample_input = {
    'loan_id': 1,
    'no_of_dependents': 2,
    'education': '12th',
    'self_employed': 'No',
    'employment_type': 'Salaried',
    'income_annum': 9600000,
    'loan_amount': 29900000,
    'loan_term': 12,
    'cibil_score': 778,
    'residential_assets_value': 2400000,
    'commercial_assets_value': 17600000,
    'luxury_assets_value': 22700000,
    'bank_asset_value': 8000000
}



In [5]:
LOAN_APPROVER_MODEL(sample_input)

/home/aditya_bendapudi/.conda/envs/aditya_bendapudi/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Real_Estate_Assets
Feature names seen at fit time, yet now missing:
- Real_Estate_Assests

  warnings.warn(message, FutureWarning)


'Approved'